The landsat_hires netcdf file can be found at  http://glasslab.engr.ccny.cuny.edu/u/hannah/eas213/landsat_hires.nc

If you have the file on your computer, set the path to the location on disk:
```python
datapath = "/path/to/landsat_hires.nc"
```
if you are running the file via mybinder (the interactive version), then set the path to the managed version of the data:
```python
from quilt.data.story645.eas213 import landsat_hires
datapath = landsat_hires._data()
```
You can also install quilt and the data package on your computer to access the data. To do so, enter the following commands in a terminal or anaconda command prompt:
```bash
pip install quilt
quilt install story645/eas213
```

In [ ]:
#set datapath variable in this cell
datapath = 

In [ ]:
import xarray as xr
ds = xr.open_dataset(datapath)

In [ ]:
ds

# Let's look at that variation a bit more:

In [ ]:
import landsat_sample_functions as lf
tempK = lf.BT(ds['Thermal Radiances'], 10.9, 1)
tempF = 9/5*(tempK - 273) + 32

In [ ]:
seasonsT = tempF.groupby('time.season').mean('time')
seasonsT.coords

In [ ]:
# http://xarray.pydata.org/en/stable/plotting.html
_ = seasonsT.plot(x='x', y='y', col='season', col_wrap=2,
                 cmap='coolwarm',vmin=0, vmax=100, 
                 figsize=(5,5))

In [ ]:
seasonsS = tempF.groupby('time.season').mean(['x', 'y'])
seasonsS.coords

In [ ]:
_ = seasonsS.plot(marker='o')

In [ ]:
tempFdf = tempF.to_dataframe()['Thermal Radiances'].to_frame().reset_index()
tempFdf.head()

![boxplot](figs/boxplot.svg)
By Jhguch at en.wikipedia, CC BY-SA 2.5, https://commons.wikimedia.org/w/index.php?curid=14524285

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np 

In [ ]:
#750615 - lat*lon
# take out this line in your program
labels = tempFdf['time'][::750615].dt.strftime("%d/%m/%y").tolist()

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
g = sns.boxplot(x = 'time', y='Thermal Radiances', 
                data=tempFdf, ax=ax)
_ = ax.set_ylabel("Brightness Temperature (F)")
#this line sets the labels created above, remove if no labels
_ = g.set_xticklabels(labels, rotation=45) 
fig.savefig("tempf_boxplot.png")


# NDVI:
Using the july 17a and july17b notebooks as reference:
1. compute NDVI
2. compute the temporal and spatial means
3. compute the monthly NDVI
4. compute the seasonal NDVI

In [ ]:
ds

In [ ]:
ds['Thermal Radiances']

In [ ]:
ds['UV']

In [ ]:
ds['Red']

In [ ]:
ds['NIR'].coords

In [ ]:
ds['NIR'].sel(time='2014-04-26').plot(cmap='spectral')

In [ ]:
#grid spec aligns subplots so that the 2nd is twice as big as the first
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,5),
                               gridspec_kw={'width_ratios':[1,2]})
_ = tempF.mean('time').plot(ax=ax1, cmap='coolwarm')
_ = tempF.mean(['x','y']).plot(ax=ax2, marker='o')